In [1]:
import pandas as pd
from datetime import datetime, timedelta
import random

STEEL_INVENTORY_NAME_LIST = [
    "山西宏达钢铁有限公司厂库",
    "山西华鑫源钢铁集团有限公司厂库",
    "安阳市新普钢铁有限公司厂库",
    "安钢集团信阳钢铁有限责任公司厂库",
    "四川德润钢铁集团航达钢铁有限责任公司厂库",
    "山西晋南钢铁集团有限公司厂库",
    "山西新泰钢铁有限公司厂库",
    "云南曲靖呈钢钢铁(集团)有限公司厂库",
    "六安钢铁控股集团有限公司仓库",
    "山西建龙实业有限公司厂库",
    "中拓物流(武汉建投汉阳库)",
]

COAL_INVENTORY_NAME_LIST = [
    "杨桥畔华茂二号库",
    "靖江盈利港",
    "京唐港",
    "直发港",
    "太仓港",
]

COMPANY_NAME_LIST = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    "湖北国贸农产品有限公司",
    "武汉鼎联丰国际贸易有限公司",
    "湖北国贸农产品有限公司武汉分公司",
    "湖北南方大集实业有限公司",
    "湖北南方大集实业有限公司东西湖分公司",
    "湖北南方大集实业有限公司慈惠分公司",
    "湖北南方大集实业有限公司江汉分公司",
    "湖北南方大集实业有限公司能源分公司",
    "湖北南方工贸有限公司",
    "湖北南方集团有限公司",
    "湖北国贸供应链管理有限公司",
    "湖北华中能源发展有限公司",
    "湖北国贸汽车有限公司红安分公司",
]

# 假定的行业列表
industries = ["煤炭", "钢材", "农产品", "汽车", "能源化工"]

# 初始化一个空列表来存储数据
data = []

# 模拟2023年的数据
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 8, 26)

# 计算总交易次数（每家公司每月平均4次交易）
total_transactions = len(COMPANY_NAME_LIST) * 4 * 12

for _ in range(total_transactions):
    # 随机选择公司名称
    company = random.choice(COMPANY_NAME_LIST)
    # 随机生成交易日期
    transaction_date = start_date + timedelta(
        days=random.randint(0, (end_date - start_date).days)
    )
    # 随机选择行业
    industry = random.choice(industries)
    # 随机生成下游客户名称
    customer_name = f"客户{random.randint(1000, 9999)}"
    # 随机生成金额
    SALES = round(random.uniform(10000, 5000000), 1)

    # 将生成的数据添加到列表
    data.append(
        {
            "company_name": company,
            "customer_name": customer_name,
            "SALES": SALES,
            "date": transaction_date.strftime("%Y-%m-%d"),
            "industry": industry,
        }
    )


# 使用列表构造DataFrame
sales_df = pd.DataFrame(data)
sales_df["date"] = pd.to_datetime(sales_df["date"])
sales_df["SALES"] = pd.to_numeric(sales_df["SALES"])
# 打印DataFrame的前几行
print(sales_df.head())

   company_name customer_name      SALES       date industry
0  湖北国贸能源化工有限公司        客户5420   732764.2 2023-09-08      农产品
1  湖北南方大集实业有限公司        客户4770  3377499.3 2023-07-04     能源化工
2    湖北国贸汽车有限公司        客户6462   278343.7 2023-03-19       钢材
3    湖北南方集团有限公司        客户5966  4722461.1 2023-09-19       煤炭
4   湖北国贸农产品有限公司        客户9799  3776104.4 2024-08-13     能源化工


In [2]:
import sys
sys.path.append('..') 
from query import SalesQueryHelper
from entity import PerformanceQuerySchema


helper = SalesQueryHelper(sales_df)

# 创建查询参数
params = PerformanceQuerySchema(
    indicator='SALES',
    aggregation='YEAR',
    start_time='2023-01-01',
    end_time='2023-12-31',
    scope='GROUP',
    operator='>',
    value='80000000',
    company_name="None"  # 如果scope为GROUP，则可能不需要company_name
)

# 执行查询
result = helper.execute_query(params)
print(result)

         company_name  year        SALES
4   湖北南方大集实业有限公司慈惠分公司  2023  103116025.2
6   湖北南方大集实业有限公司能源分公司  2023   88091479.5
7          湖北南方工贸有限公司  2023   88815031.3
12         湖北国贸汽车有限公司  2023   86049023.2


In [3]:
params_by_month=PerformanceQuerySchema(
    indicator='SALES',
    aggregation='YEAR',
    start_time='2023-01-01',
    end_time='2023-12-31',
    scope='COMPANY',
    company_name="湖北国贸能源化工有限公司"  # 如果scope为GROUP，则可能不需要company_name
)
result = helper.execute_query(params_by_month)
print(result)

   company_name  year       SALES
0  湖北国贸能源化工有限公司  2023  68407461.2


In [4]:
import csv
import json

# 存储 Result 列的数据
results = []

# 打开 CSV 文件并读取内容
with open('../test/output.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    # 遍历每一行，提取 Result 列的数据
    for row in csvreader:
        results.append(row['Result'].replace("'", '"'))

# 打印 Result 列的数据以验证结果
data_results=[]
for query in results:
    try:
        # json_data = json.loads(query)
        params=PerformanceQuerySchema(**eval(query))
        if params.indicator == "SALES":
            print(params)
            data = helper.execute_query(params)
            data_results.append(data)
            print(data)
    except Exception as e:
        print(e)


indicator='SALES' aggregation='YEAR' start_time='2024-01-01' end_time='2024-12-31' scope='GROUP' sort_type='DESC' operator=None value=None company_name=None
          company_name  year       SALES
0        武汉鼎联丰国际贸易有限公司  2024  48277983.5
1         湖北华中能源发展有限公司  2024  44096354.7
2         湖北南方大集实业有限公司  2024  31783572.8
3   湖北南方大集实业有限公司东西湖分公司  2024  33676276.5
4    湖北南方大集实业有限公司慈惠分公司  2024  58605430.9
5    湖北南方大集实业有限公司江汉分公司  2024  33316581.2
6    湖北南方大集实业有限公司能源分公司  2024  49384693.4
7           湖北南方工贸有限公司  2024  31333043.2
8           湖北南方集团有限公司  2024  39026487.9
9        湖北国贸供应链管理有限公司  2024  61494324.7
10         湖北国贸农产品有限公司  2024  38685710.4
11    湖北国贸农产品有限公司武汉分公司  2024  57377922.9
12          湖北国贸汽车有限公司  2024  57710527.2
13     湖北国贸汽车有限公司红安分公司  2024  37318654.3
14        湖北国贸能源化工有限公司  2024  48300935.7
15        湖北国贸金属矿产有限公司  2024  39468736.3
16        湖北国际贸易集团有限公司  2024  66028788.5
indicator='SALES' aggregation='YEAR' start_time='2023-01-01' end_time='2023-12-31' scope='GROUP' sort_typ

/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["month"] = filtered_df["date"].dt.to_period("M")
/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["month"] = filtered_df["date"].dt.to_period("M")
/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [5]:
from plot import SalesBarChart,SalesBarChartForGroup


def fig_generator():
    for data in data_results:
        df = pd.DataFrame(data)
        if "month" in df.columns:
            chart=SalesBarChartForGroup(df)
        else:
            chart = SalesBarChart(df)
        chart.plot()
        yield chart.fig

for fig in fig_generator():
    fig.show()

Index(['company_name', 'month', 'SALES'], dtype='object')


Index(['company_name', 'month', 'SALES'], dtype='object')


Index(['company_name', 'month', 'SALES'], dtype='object')
